## Meta Ensemble

Now we are going to use the random forest classifiers trained above to create a meta predictor.

In [ ]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
train.head()

In [ ]:
X_train, Y_train = train.ix[:,:-1], train['TARGET']

In [ ]:
X_train.head()

In [ ]:
for rf in rfs:
    temp = rf.predict(train.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_train = pd.concat([X_train, temp], axis=1)

In [ ]:
train_meta = pd.concat([X_train, train['TARGET']], axis=1)

In [ ]:
train_meta.head()

In [ ]:
X_test, Y_test = test.ix[:,:-1], test['TARGET']

In [ ]:
for rf in rfs:
    temp = rf.predict(test.ix[:,:-1])
    temp = pd.DataFrame(temp)
    X_test = pd.concat([X_test, temp], axis=1)

In [ ]:
Y_test = test['TARGET']

In [ ]:
X_test.head()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# random regularization
cv = {}

In [ ]:
for _ in range(20):
    n_trees = np.random.randint(50,300)
    lg = RandomForestClassifier(n_estimators = n_trees)
    lg.fit(X_train, Y_train)
    cv[n_trees] = roc_auc_score(Y_test, lg.predict(X_test))
    print(cv[n_trees])

In [ ]:
for key in sorted(cv.keys(), key=cv.get)[::-1]:
    print(key,cv[key])

### Score analysis

In [ ]:
X_test, Y_test = test.ix[:,:-1], test.ix[:,-1]
n = 100
a = 0.25
w = 1
N_forest = 5
n_trees = 5

In [ ]:
scores = []
for _ in range(n):
    rfs = trainForests(train, a, w, N_forest,n_trees)
    Y_prob = mean_ensemble(rfs, X_test)
    scores.append(roc_auc_score(Y_test,Y_prob))
scores = pd.DataFrame(scores)

In [ ]:
scores.describe()

In [ ]:
plt.title("Distribution of scores")
plt.hist(scores)
plt.show()

In [ ]:
# If desired, transform probabilities into class labels.
def threshold(Y_prob, threshold = 0.5):
    result = []
    for y in Y_prob:
        if y <= threshold:
            result.append(0)
        else:
            result.append(1)
    return result

In [ ]:
# Evaluate class labels
Y_pred = threshold(Y_prob, threshold = 0.5)
_ = eval_classification(test['TARGET'],Y_pred, print_results = True)

In [ ]:
# Plot feature importance
def plot_features(forest):  
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    n=len(indices)
    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(n), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(n), indices)
    plt.xlim([-1, n])
    plt.show()

In [ ]:
train.head()

# Create Submission

In [ ]:
# Retrain forest on the whole 'train.csv' data
rfs = trainForests(data, a, w, N_forest, n_trees)

In [ ]:
test = pd.read_csv('data/test.csv')
test_id = test.ix[:,'ID'].values
process(test)

In [ ]:
Y_prob = mean_ensemble(rfs,test)

In [ ]:
create_submission(test_id, Y_prob['geometric'])

## Ensemble RF and XGBOOST

In [ ]:
Y_boost = pd.read_csv('../Kaggle_Santander-master/simplexgbtest.csv')

In [ ]:
Y_boost.head()

In [ ]:
Y_rf = pd.read_csv('submissions/rforest_ensemble2.csv')

In [ ]:
Y_rf.head()

In [ ]:
Y_prob = pd.concat([Y_boost,Y_rf.ix[:,'TARGET']], axis=1, ignore_index=True)

In [ ]:
Y_prob.rename(columns ={0:'ID', 1:'xgb', 2: 'rfe' }, inplace = True)

In [ ]:
# geometric mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob.ix[:,1:].product(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['geometric'] = temp

In [ ]:
# arithmetic mean ensemble
l = 2 #number of predictors to ensemble
temp = Y_prob[['xgb', 'rfe']].mean(axis=1)
temp = temp.apply(lambda x: np.power(x, 1./l))
Y_prob['arithmetic'] = temp

In [ ]:
# difference column
temp = Y_prob['xgb'] - Y_prob['rfe']
Y_prob['xgb - rfe'] = temp

In [ ]:
# difference column
temp = Y_prob['geometric'] - Y_prob['arithmetic']
Y_prob['geo - ari'] = temp

In [ ]:
Y_prob.head()

In [ ]:
plt.title('Differences between XGB and RFE')
plt.hist(Y_prob['xgb - rfe'])
plt.show()

In [ ]:
plt.title('Differences between ensembles')
plt.hist(Y_prob['geo - ari'])
plt.show()

In [ ]:
create_submission(test_id, Y_prob['arithmetic'])